# 대형아울렛 

통계청에서 정의한 대형아울렛 입지유형 -> 전국 대형 쇼핑몰, 아울렛이 소재한 행정동 😎  
유통업체 현황 데이터 사용

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import matplotlib
from numpy import inf

matplotlib.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'Malgun Gothic'
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
%matplotlib inline

## 서울
http://data.seoul.go.kr/dataList/10128/S/2/datasetView.do (이 데이터에서 컬럼 이름을 일부 수정하였습니다)  
chrome 말고 internet explorer로 들어가야 오류가 나지 않습니다

In [2]:
seoul = pd.read_table("data/서울시 유통업체현황 동별 통계.txt", header=3, sep='\t')
seoul = seoul[['자치구', '동', '대형마트 개소', '백화점 개소', '전문점 개소', '쇼핑센터 개소', '복합쇼핑몰 개소', '그밖의 대규모점포 개소']]

# 중구 / 노원구만
seoul = seoul.query('자치구 == "중구" or 자치구=="노원구"').query('동 != "소계"').reset_index(drop=True)

# '-'값을 0으로 대체합니다
seoul = seoul.replace('-', 0)

# 수치형으로
for column in seoul.columns[2:]:
    seoul[column] = seoul[column].map(int)
    
# 대형아울렛 입지유형은 대형마트/백화점/쇼핑센터/복합쇼핑몰 존재 유무에 따라 나누도록 하겠습니다
seoul['합계'] = seoul[['대형마트 개소', '백화점 개소', '쇼핑센터 개소', '복합쇼핑몰 개소']].sum(axis=1)

# 대구데이터와 합칠 예정이므로 시도 정보를 붙여줍니다
seoul['시도'] = "서울특별시"

## 대구
- 수성구 : http://kosis.kr/statHtml/statHtml.do?orgId=556&tblId=DT_55601_H001001
- 중구 : http://kosis.kr/statHtml/statHtml.do?orgId=551&tblId=DT_55101_H001001

In [3]:
daegu = pd.read_csv("data/대구 유통업체현황.csv")

# '-'값을 0으로 대체합니다
daegu = daegu.replace('\xa0-', 0)

# 수치형으로
for column in daegu.columns[2:]:
    daegu[column] = daegu[column].map(int)

# 대형아울렛 입지유형은 대형마트/백화점/쇼핑센터/복합쇼핑몰 존재 유무에 따라 나누도록 하겠습니다
daegu['합계'] = daegu[['대형마트(할인점) 업체수', '백화점 업체수', '쇼핑센터 업체수', '복합쇼핑몰']].sum(axis=1)

# 서울데이터와 합칠 예정이므로 시도 정보를 붙여줍니다
daegu['시도'] = "대구광역시"

# 서울데이터와 합치므로 컬럼이름 일치시켜 줍니다
daegu.rename(columns={"구":"자치구", 
                      "대형마트(할인점) 업체수":"대형마트 개소", 
                      "전문점 업체수":"전문점 개소",
                      "백화점 업체수":"백화점 개소", 
                      "쇼핑센터 업체수":"쇼핑센터 개소", 
                      "기타대규모 점포 업체수":"그밖의 대규모점포 개소", 
                      "복합쇼핑몰":"복합쇼핑몰 개소"}, inplace=True)

## 최종

In [4]:
seoul = seoul[["시도", "자치구", "동", '대형마트 개소', '백화점 개소', '쇼핑센터 개소', "복합쇼핑몰 개소", "합계"]]
daegu = daegu[["시도", "자치구", "동", '대형마트 개소', '백화점 개소', '쇼핑센터 개소', "복합쇼핑몰 개소", "합계"]]

In [5]:
seoul_daegu = pd.concat([seoul, daegu], axis=0).reset_index(drop=True)

In [6]:
seoul_daegu["outlet_region"] = seoul_daegu['합계'].apply(lambda x:x>0)

In [7]:
seoul_daegu

# 서울특별시 일부 동은 안나와있는데 http://data.seoul.go.kr/dataList/10128/S/2/datasetView.do 에 보면
# 개소가 1 이상인 동만 나와있다.
# 즉, 대형마트, 백화점, 쇼핑센터, 복합쇼핑몰 중 아무것도 없어서 기재되지 않았다는 뜻 아닐까?

,시도,자치구,동,대형마트 개소,백화점 개소,쇼핑센터 개소,복합쇼핑몰 개소,합계,outlet_region
0,서울특별시,중구,소공동,0,2,0,0,2,True
1,서울특별시,중구,회현동,1,1,2,0,4,True
2,서울특별시,중구,명동,0,0,1,0,1,True
3,서울특별시,중구,광희동,0,0,4,0,4,True
4,서울특별시,중구,을지로동,0,0,0,0,0,False
5,서울특별시,중구,황학동,1,0,0,0,1,True
6,서울특별시,중구,신당동,0,0,1,0,1,True
7,서울특별시,중구,약수동,0,0,0,0,0,False
8,서울특별시,노원구,월계3동,0,0,1,0,1,True
9,서울특별시,노원구,하계1동,0,0,0,0,0,False


In [140]:
seoul_daegu.to_csv("data/서울_대구_대형아울렛지역_정의.csv", index=False)

In [ ]:
seoul_daegu.query('outlet_region == True')[['시도', '자치구', '동']]

### merge

In [9]:
region = pd.read_csv("region.csv")

In [12]:
region.head()

,HDONG_NM,sport_leisure,commercial,resident,직장인구비율
0,소공동,0,1,0,1
1,회현동,0,1,0,1
2,명동,0,1,0,1
3,필동,0,0,0,1
4,장충동,1,0,0,1


In [26]:
seoul_daegu['동'] = seoul_daegu['동'].apply(lambda x:x.replace('\xa0', ''))

In [27]:
temp = pd.merge(region, seoul_daegu[['동', 'outlet_region']],
                left_on = 'HDONG_NM',
                right_on = '동',
                how='left')

In [36]:
del temp['동']

In [44]:
temp['outlet_region'] = temp['outlet_region'].apply(lambda x:1 if x==True else 0)

In [46]:
temp.rename(columns={'직장인구비율':'work', 
                     'outlet_region':'outlet'}, inplace=True)

In [53]:
temp.to_csv("region.csv", encoding='cp949', index=False)

In [51]:
(pd.read_pickle("레저_상업_주거지구구분.pickle")['resident'] == temp['resident']).sum()

69